In [1]:
import math
from math import log2
import re 
import numpy as np
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import shutil
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

##### define different functions of calculating differential score
def ks_score(r1, r2):
    r = r1-r2
    max_ks = max(max(r), 0)
    min_ks = min(min(r), 0)
    return (abs(max_ks)+abs(min_ks))/r1.sum()

def entropy_score(r_true:[],r_false:[]):

    h = 0.0
    for i in range(0,len(r_true)):
        if r_false[i]!=0:
            h = -r_true[i] * log2(r_false[i])+h
        else:
            h = -r_true[i] * log2(1/100000) + h
    return h
def KL(p,q):
    return sum(_p*math.log(_p/_q) for (_p,_q) in zip(p,q) if _p != 0 )

def JS(p,q):
    M = [0.5*(_p +_q) for (_p,_q) in zip(p,q)]
    return 0.5*(KL(p,M)+KL(q,M))



In [2]:
def read_d(dir_name,sim_type):######this function is uesd to collect the results of each bit.
    
    def read_file(name):
        aa=pd.read_csv(name)
        getcol=[columns for columns in aa]
        del aa
        temp=[]
        for c in getcol:
            temp.append(remove(c))
        return dict(temp)

    def remove(text):
        punctuation = '[{} ？!,;+"\'' ####the result is saved in csv file and there are punctuations in the front and end of the rusult.
        text = re.sub(r'[{}]+'.format(punctuation), '', text)
        text=re.split(":",text)
        return text
    files=os.listdir(str(dir_name))
    whole_list=[]
    result=[]
    name=[]
    for file in files:
        if  sim_type.search(file):
            name.append(file)
            result.append(read_file(dir_name+"/"+file))
    temp_len=[]
    for i in result:
        temp_len.append(len(i))
    full_dic=result[temp_len.index(max(temp_len))].keys()  ####some of the file lack some keys for no shots on it. Find the full key as a standard example
    for dic in result:
        if len(full_dic - dic.keys()) != 0:
            for key in full_dic - dic.keys():##fulfill all lack keys
                dic[key]=0
    for ele in result:
        tem=sorted(list(ele.items() ), key=lambda x:x[0]) #####set the results in ascending order of key 
        pp=[]
        for i in tem:
            pp.append(  float(i[1]) )
        whole_list.append(pp)#####save the name and result together
    return whole_list,name

In [3]:
#####set the type of backend to collect
noise_file = re.compile("startPyquil_statevector_[0-9]+"+".csv") 

In [4]:
#####JS score with standardised answers
def formal_js(a,b):
    thershold=0.5
    candidates = [] # save right answer candidates
    answer = -1
    for results in a:
        flag = -1
        for i in range(0,len(candidates)):
            if JS(candidates[i]/(candidates[i].sum()),np.asarray(results)/(np.asarray(results).sum()))<thershold:
                flag = i
                candidates[i] = candidates[i] + np.asarray(results)/(np.asarray(results).sum())
                if candidates[i].sum() > candidates[answer].sum():
                    answer = i
                break

        if flag == -1:
            candidates.append(np.asarray(results)/(np.asarray(results).sum()))
            if answer == -1:
                answer = 0

    wrong_out = []           
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)

    result=[]

    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
            
    return result,wrong_out


js_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_error.append(["folder"+str(i),formal_js(a,b)])
for i in range(0,len(js_error)):
    if len(js_error[i][1][1])>0:
        print(js_error[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 2.22608696e+02 3.33913043e+01
 3.77281302e-30 0.00000000e+00 3.77281302e-30 0.00000000e+00
 3.77281302e-30 0.00000000e+00 3.77281302e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.22608696e+02 3.33913043e+01]
reading folder2
------------------------------------------
Right answer: [8.40308356e-31 0.00000000e+00 8.40308356e-31 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.33739130e+02 2.22608696e+01
 2.56000000e+02 0.00000000e+00 2.33739130e+02 2.22608696e+01
 8.40308356e-31 0.00000000e+00 8.40308356e-31 0.00000000e+00]
reading folder3
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 2.44869565e+02 1.11304348e+01
 9.60352406e-31 0.00000000e+00 9.60352406e-31 0.00000000e+00
 9.60352406e-31 0.00000000e+00 9.60352406e-31 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.44869565e+02 1.11304348e+01]
reading folder4
------------------------------

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.62953635e-30 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.02400000e+03 0.00000000e+00]
0.6931471805599453
startPyquil_statevector_211.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:0.6931471805599453
0.6931471805599453
startPyquil_statevector_212.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:0.6931471805599453
0.6931471805599453
startPyquil_statevector_213.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:0.6931471805599453
0.6931471805599453
startPyquil_statevector_214.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999987]:0.6931471805599453
0.6931471805599453
startPyquil_statevector_215.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Right answer: [2.56000000e+02 0.00000000e+00 2.11478261e+02 4.45217391e+01
 4.08149773e-30 0.00000000e+00 4.08149773e-30 0.00000000e+00
 4.08149773e-30 0.00000000e+00 4.08149773e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.11478261e+02 4.45217391e+01]
reading folder52
------------------------------------------
Right answer: [ 69.88921603 186.11078397  95.15477428 160.84522572   0.
   0.           0.           0.          69.88921603 186.11078397
  95.15477428 160.84522572   0.           0.           0.
   0.        ]
reading folder53
------------------------------------------
Right answer: [2.56000000e+02 4.45217391e+01 0.00000000e+00 2.11478261e+02
 6.85966005e-32 0.00000000e+00 0.00000000e+00 6.85966005e-32
 6.85966005e-32 0.00000000e+00 0.00000000e+00 6.85966005e-32
 2.56000000e+02 4.45217391e+01 0.00000000e+00 2.11478261e+02]
reading folder54
------------------------------------------
Right answer: [ 69.88921603 186.11078397  74.94232768 181.05767232  69.88921603
 186.110783

Right answer: [  0.           0.           0.           0.         186.11078397
  69.88921603 115.36722088 140.63277912 186.11078397  69.88921603
 115.36722088 140.63277912   0.           0.           0.
   0.        ]
reading folder81
------------------------------------------
Right answer: [34.94460802 93.05539198 93.05539198 34.94460802 34.94460802 93.05539198
 93.05539198 34.94460802 34.94460802 93.05539198 93.05539198 34.94460802
 34.94460802 93.05539198 93.05539198 34.94460802]
reading folder82
------------------------------------------
Right answer: [1.55792114e+02 1.00207886e+02 1.16055327e-30 7.20769129e-31
 1.22914987e-30 6.52172528e-31 1.86110784e+02 6.98892160e+01
 1.22914987e-30 6.52172528e-31 1.86110784e+02 6.98892160e+01
 1.55792114e+02 1.00207886e+02 1.16055327e-30 7.20769129e-31]
reading folder83
------------------------------------------
Right answer: [ 41.57009369  23.26788556  59.72023178 119.828887   136.52573567
  76.41708046  18.18393886  36.48614698 136.52573567

In [5]:
##JS score with trimmed mean 
def js_ternn(a,b):
    thershold=0.6
    candidates = [] # save right answer candidates
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


js_ter=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_ter.append(["folder"+str(i),js_ternn(a,b)])
for i in range(0,len(js_ter)):
    if len(js_ter[i][1][1])>0:
        print(js_ter[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00
 3.47098798e-30 0.00000000e+00 3.47098798e-30 0.00000000e+00
 3.47098798e-30 0.00000000e+00 3.47098798e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00]
reading folder2
------------------------------------------
Right answer: [1.05181454e-31 0.00000000e+00 1.05181454e-31 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00
 1.05181454e-31 0.00000000e+00 1.05181454e-31 0.00000000e+00]
reading folder3
------------------------------------------
Right answer: [256.   0. 256.   0.   0.   0.   0.   0.   0.   0.   0.   0. 256.   0.
 256.   0.]
reading folder4
------------------------------------------
Right answer: [  3.92196321   0.           3.660499     0.26146421 252.07803679
   0.         235.27283434  16.80520245 252.07803679   0.
 235.272834

Right answer: [93.05539198 34.94460802 93.05539198 34.94460802 93.05539198 34.94460802
 93.05539198 34.94460802 93.05539198 34.94460802 93.05539198 34.94460802
 93.05539198 34.94460802 93.05539198 34.94460802]
reading folder33
------------------------------------------
Right answer: [186.11078397  69.88921603  85.38542509 170.61457491   0.
   0.           0.           0.           0.           0.
   0.           0.         186.11078397  69.88921603  85.38542509
 170.61457491]
reading folder34
------------------------------------------
Right answer: [93.05539198 34.94460802 93.05539198 34.94460802 93.05539198 34.94460802
 93.05539198 34.94460802 93.05539198 34.94460802 93.05539198 34.94460802
 93.05539198 34.94460802 93.05539198 34.94460802]
reading folder35
------------------------------------------
Right answer: [132.54973116   0.           0.         132.54973116 196.24596745
   0.           0.         196.24596745  59.75403255   0.
   0.          59.75403255 123.45026884   0.       

Right answer: [5.43699331e-31 3.60925690e-32 0.00000000e+00 0.00000000e+00
 3.72221568e+02 8.38670592e+01 5.59113728e+01 8.52974197e-31
 3.72221568e+02 8.38670592e+01 5.59113728e+01 8.52974197e-31
 4.51411336e-31 3.60925690e-32 0.00000000e+00 0.00000000e+00]
reading folder62
------------------------------------------
Right answer: [0.00000000e+00 6.31088724e-31 0.00000000e+00 4.20725816e-31
 0.00000000e+00 2.56000000e+02 1.70666667e+01 2.38933333e+02
 0.00000000e+00 2.56000000e+02 1.70666667e+01 2.38933333e+02
 0.00000000e+00 6.31088724e-31 0.00000000e+00 4.20725816e-31]
reading folder63
------------------------------------------
Right answer: [ 69.88921603 186.11078397 139.62215679 116.37784321   0.
   0.           0.           0.          69.88921603 186.11078397
 139.62215679 116.37784321   0.           0.           0.
   0.        ]
reading folder64
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 0.00000000e+00 2.56000000e+02
 1.36533333e+02 

Right answer: [ 69.88921603 186.11078397 139.62215679 116.37784321  69.88921603
 186.11078397 139.62215679 116.37784321   0.           0.
   0.           0.           0.           0.           0.
   0.        ]
reading folder92
------------------------------------------
Right answer: [284.44444444   0.           0.         132.74074074 227.55555556
   0.           0.          56.88888889   0.          94.81481481
   0.           0.           0.         227.55555556   0.
   0.        ]
reading folder93
------------------------------------------
Right answer: [1.87529729e+02 7.04220652e+01 1.12517838e+02 4.22532391e+01
 2.16902707e-31 7.94875336e-32 6.72047141e+01 2.81688261e+01
 2.16902707e-31 7.94875336e-32 7.50118917e+01 2.81688261e+01
 1.87529729e+02 7.04220652e+01 1.12517838e+02 4.22532391e+01]
reading folder94
------------------------------------------
Right answer: [232.47487486   0.43345844  99.23396275   0.99092435  56.45276772
   0.43345844 120.98962908   0.99092435  56.4527677

In [6]:
#JS score with 2-means
def js_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=0.5
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


js_2means=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_2means.append(["folder"+str(i),js_2mean(a,b)])
for i in range(0,len(js_2means)):
    if len(js_2means[i][1][1])>0:
        print(js_2means[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 3.33748845e-30 0.00000000e+00 3.33748845e-30 0.00000000e+00
 3.33748845e-30 0.00000000e+00 3.33748845e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02]
reading folder2
------------------------------------------
Right answer: [2.77416085e-30 0.00000000e+00 2.77416085e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 2.77416085e-30 0.00000000e+00 2.77416085e-30 0.00000000e+00]
reading folder3
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 7.27903472e-30 0.00000000e+00 5.16516069e-30 0.00000000e+00
 5.16516069e-30 0.00000000e+00 5.16516069e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02]
reading folder4
------------------------------

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.15544362e-30 1.26217745e-29
 0.00000000e+00 0.00000000e+00 5.12000000e+02 5.12000000e+02]
reading folder30
------------------------------------------
Right answer: [ 69.88921603 186.11078397 128.         128.           0.
   0.           0.           0.          69.88921603 186.11078397
 128.         128.           0.           0.           0.
   0.        ]
reading folder31
------------------------------------------
Right answer: [0.00000000e+00 0.00000000e+00 2.56000000e+02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.41230079e-29 0.00000000e+00
 0.00000000e+00 2.21768522e-29 2.21768522e-29 2.41230079e-29
 0.00000000e+00 2.56000000e+02 2.56000000e+02 2.56000000e+02]
reading folder32
------------------------------------------
Right answer: [93.05539198 34.94460802 64.         64.         93.05539198 34.94460802
 64.

Right answer: [5.12000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.17655317e-29 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.17655317e-29 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.12000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
reading folder59
------------------------------------------
Right answer: [512.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 512.   0.
   0.   0.]
reading folder60
------------------------------------------
Right answer: [5.1200000e+02 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.7710561e-30
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.7710561e-30 0.0000000e+00
 0.0000000e+00 0.0000000e+00 5.1200000e+02 0.0000000e+00 0.0000000e+00
 0.0000000e+00]
reading folder61
------------------------------------------
Right answer: [2.47498330e-30 1.33441239e-30 7.88860905e-31 3.12918666e-61
 3.72221568e+02 6.98892160e+01 6.98892160e+01 1.05622369e-29
 3.72221568e+02 6.98892160e+01 6.98892160e+01 1.23757700e-29
 2.58135886e-30 2.262614

Right answer: [ 24.44582472  12.22291236   0.          12.22291236 231.55417528
 115.77708764   0.         115.77708764 231.55417528 115.77708764
   0.         115.77708764  24.44582472  12.22291236   0.
  12.22291236]
reading folder88
------------------------------------------
Right answer: [1.52387686e-30 2.47297514e-30 2.50411439e-30 1.40570631e-30
 6.98892160e+01 1.28000000e+02 1.86110784e+02 1.28000000e+02
 6.98892160e+01 1.28000000e+02 1.86110784e+02 1.28000000e+02
 1.52387686e-30 2.47297514e-30 2.50411439e-30 1.40570631e-30]
reading folder89
------------------------------------------
Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.12000000e+02 0.00000000e+00 2.56000000e+02 2.56000000e+02
 7.88860905e-31 0.00000000e+00 7.88860905e-31 0.00000000e+00]
reading folder90
------------------------------------------
Right answer: [93.05539198 34.94460802 64.         64.         93.05539198 34.94460

In [7]:
####KS score with standardised answers
def formal_ks(a,b):
    thershold=0.8
    candidates = [] # save right answer candidates
    answer = -1
    for results in a:
        flag = -1
        for i in range(0,len(candidates)):
            if ks_score(candidates[i]/(candidates[i].sum()),np.asarray(results)/(np.asarray(results).sum()))<thershold:
                flag = i
                candidates[i] = candidates[i] + np.asarray(results)/(np.asarray(results).sum())
                if candidates[i].sum() > candidates[answer].sum():
                    answer = i
                break

        if flag == -1:
            candidates.append(np.asarray(results)/(np.asarray(results).sum()))
            if answer == -1:
                answer = 0

    wrong_out = []           
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)

    result=[]

    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
            
    return result,wrong_out

ks_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_error.append(["folder"+str(i),formal_ks(a,b)])
for i in range(0,len(ks_error)):
    if len(ks_error[i][1][1])>0:
        print(ks_error[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 2.22608696e+02 3.33913043e+01
 3.77281302e-30 0.00000000e+00 3.77281302e-30 0.00000000e+00
 3.77281302e-30 0.00000000e+00 3.77281302e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.22608696e+02 3.33913043e+01]
reading folder2
------------------------------------------
Right answer: [8.40308356e-31 0.00000000e+00 8.40308356e-31 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.33739130e+02 2.22608696e+01
 2.56000000e+02 0.00000000e+00 2.33739130e+02 2.22608696e+01
 8.40308356e-31 0.00000000e+00 8.40308356e-31 0.00000000e+00]
reading folder3
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 2.44869565e+02 1.11304348e+01
 9.60352406e-31 0.00000000e+00 9.60352406e-31 0.00000000e+00
 9.60352406e-31 0.00000000e+00 9.60352406e-31 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.44869565e+02 1.11304348e+01]
reading folder4
------------------------------

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.62953635e-30 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.02400000e+03 0.00000000e+00]
2.0
startPyquil_statevector_211.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:2.0
2.0
startPyquil_statevector_212.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:2.0
2.0
startPyquil_statevector_213.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:2.0
2.0
startPyquil_statevector_214.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999987]:2.0
2.0
startPyquil_statevector_215.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:2.0
2.0
startPyquil_statevector_219.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

Right answer: [2.56000000e+02 0.00000000e+00 2.11478261e+02 4.45217391e+01
 4.08149773e-30 0.00000000e+00 4.08149773e-30 0.00000000e+00
 4.08149773e-30 0.00000000e+00 4.08149773e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.11478261e+02 4.45217391e+01]
reading folder52
------------------------------------------
Right answer: [ 69.88921603 186.11078397  95.15477428 160.84522572   0.
   0.           0.           0.          69.88921603 186.11078397
  95.15477428 160.84522572   0.           0.           0.
   0.        ]
reading folder53
------------------------------------------
Right answer: [2.56000000e+02 4.45217391e+01 0.00000000e+00 2.11478261e+02
 6.85966005e-32 0.00000000e+00 0.00000000e+00 6.85966005e-32
 6.85966005e-32 0.00000000e+00 0.00000000e+00 6.85966005e-32
 2.56000000e+02 4.45217391e+01 0.00000000e+00 2.11478261e+02]
reading folder54
------------------------------------------
Right answer: [ 69.88921603 186.11078397  74.94232768 181.05767232  69.88921603
 186.110783

Right answer: [186.11078397  69.88921603   0.           0.           0.
   0.         120.42033253 135.57966747   0.           0.
 120.42033253 135.57966747 186.11078397  69.88921603   0.
   0.        ]
reading folder78
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 0.00000000e+00 2.56000000e+02
 7.79130435e+01 0.00000000e+00 0.00000000e+00 7.79130435e+01
 1.78086957e+02 0.00000000e+00 0.00000000e+00 1.78086957e+02
 2.09535518e-30 0.00000000e+00 0.00000000e+00 2.09535518e-30]
reading folder79
------------------------------------------
Right answer: [ 69.88921603 186.11078397 155.79211407 100.20788593  69.88921603
 186.11078397 155.79211407 100.20788593   0.           0.
   0.           0.           0.           0.           0.
   0.        ]
reading folder80
------------------------------------------
Right answer: [  0.           0.           0.           0.         186.11078397
  69.88921603 115.36722088 140.63277912 186.11078397  69.88921603
 

In [8]:
##KS score with trimmed mean
def ks_ternn(a,b):
    thershold=0.6
    candidates = [] # save right answer candidates

    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

ks_ter=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_ter.append(["folder"+str(i),ks_ternn(a,b)])
for i in range(0,len(ks_ter)):
    if len(ks_ter[i][1][1])>0:
        print(ks_ter[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00
 3.47098798e-30 0.00000000e+00 3.47098798e-30 0.00000000e+00
 3.47098798e-30 0.00000000e+00 3.47098798e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00]
reading folder2
------------------------------------------
Right answer: [1.05181454e-31 0.00000000e+00 1.05181454e-31 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00
 1.05181454e-31 0.00000000e+00 1.05181454e-31 0.00000000e+00]
reading folder3
------------------------------------------
Right answer: [256.   0. 256.   0.   0.   0.   0.   0.   0.   0.   0.   0. 256.   0.
 256.   0.]
reading folder4
------------------------------------------
Right answer: [  3.92196321   0.           3.660499     0.26146421 252.07803679
   0.         235.27283434  16.80520245 252.07803679   0.
 235.272834

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.20725816e-31 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.46133333e+02 4.77866667e+02]
0.9333333333333341
startPyquil_statevector_210.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996, 0.0]:0.9333333333333341
1.066666666666666
startPyquil_statevector_211.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:1.066666666666666
1.066666666666666
startPyquil_statevector_212.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:1.066666666666666
1.066666666666666
startPyquil_statevector_213.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:1.066666666666666
1.066666666666666
startPyquil_statevector_214.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Right answer: [6.98892160e+01 1.86110784e+02 1.24125948e+02 1.31874052e+02
 5.25907270e-32 2.10362908e-31 5.25907270e-32 2.10362908e-31
 5.25907270e-32 2.10362908e-31 5.25907270e-32 2.10362908e-31
 6.98892160e+01 1.86110784e+02 1.24125948e+02 1.31874052e+02]
reading folder46
------------------------------------------
Right answer: [34.94460802 93.05539198 58.1889216  69.8110784  34.94460802 93.05539198
 58.1889216  69.8110784  34.94460802 93.05539198 58.1889216  69.8110784
 34.94460802 93.05539198 58.1889216  69.8110784 ]
reading folder47
------------------------------------------
Right answer: [ 69.88921603 124.12594774 186.11078397 131.87405226   0.
   0.           0.           0.          69.88921603 124.12594774
 186.11078397 131.87405226   0.           0.           0.
   0.        ]
reading folder48
------------------------------------------
Right answer: [34.94460802 54.31486934 93.05539198 73.68513066 34.94460802 54.31486934
 93.05539198 73.68513066 34.94460802 54.31486934 93.05

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.12000000e+02 2.04800000e+02 0.00000000e+00 3.07200000e+02
 6.57384088e-31 1.05181454e-31 0.00000000e+00 0.00000000e+00]
reading folder74
------------------------------------------
Right answer: [252.07803679   0.         252.07803679   0.           3.92196321
   0.           3.92196321   0.           3.92196321   0.
   3.92196321   0.         252.07803679   0.         252.07803679
   0.        ]
reading folder75
------------------------------------------
Right answer: [34.94460802 93.05539198 34.94460802 93.05539198 34.94460802 93.05539198
 34.94460802 93.05539198 54.31486934 73.68513066 69.8110784  58.1889216
 54.31486934 73.68513066 69.8110784  58.1889216 ]
reading folder76
------------------------------------------
Right answer: [ 49.06901725   7.18969784 115.55468441  73.1934262  179.46138469
  32.01688207  39.07862191  16.43628562 179.46138469 

Right answer: [ 69.88921603 186.11078397  69.88921603 186.11078397  69.88921603
 186.11078397  69.88921603 186.11078397   0.           0.
   0.           0.           0.           0.           0.
   0.        ]
folder18
folder27
folder29
folder31
folder49
folder89


In [9]:
##KS score with 2-means

def ks_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=0.6
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

 
ks_2means=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_2means.append(["folder"+str(i),ks_2mean(a,b)])
for i in range(0,len(ks_ter)):
    if len(ks_2means[i][1][1])>0:
        print(ks_2means[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 3.33748845e-30 0.00000000e+00 3.33748845e-30 0.00000000e+00
 3.33748845e-30 0.00000000e+00 3.33748845e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02]
reading folder2
------------------------------------------
Right answer: [2.77416085e-30 0.00000000e+00 2.77416085e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 2.77416085e-30 0.00000000e+00 2.77416085e-30 0.00000000e+00]
reading folder3
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 5.16516069e-30 0.00000000e+00 5.16516069e-30 0.00000000e+00
 7.27903472e-30 0.00000000e+00 5.16516069e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02]
reading folder4
------------------------------

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.15544362e-30 1.26217745e-29
 0.00000000e+00 0.00000000e+00 5.12000000e+02 5.12000000e+02]
1.0000000000000009
startPyquil_statevector_210.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996, 0.0]:1.0000000000000009
0.9999999999999993
startPyquil_statevector_211.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:0.9999999999999993
0.9999999999999993
startPyquil_statevector_212.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]:0.9999999999999993
0.9999999999999993
startPyquil_statevector_213.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999996]:0.9999999999999993
0.9999999999999993
startPyquil_statevector_214.csv:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

Right answer: [ 50.66206312   9.09196943 127.98309962  68.26286783 166.38585156
  29.86011589  38.9689857   20.78504684 166.38585156  29.86011589
  38.9689857   20.78504684  50.66206312   9.09196943 127.98309962
  68.26286783]
reading folder45
------------------------------------------
Right answer: [6.98892160e+01 1.86110784e+02 1.28000000e+02 1.28000000e+02
 7.47990645e-31 2.12241148e-30 7.47990645e-31 1.18329136e-30
 7.47990645e-31 1.18329136e-30 7.47990645e-31 1.18329136e-30
 6.98892160e+01 1.86110784e+02 1.28000000e+02 1.28000000e+02]
reading folder46
------------------------------------------
Right answer: [34.94460802 93.05539198 64.         64.         34.94460802 93.05539198
 64.         64.         34.94460802 93.05539198 64.         64.
 34.94460802 93.05539198 64.         64.        ]
reading folder47
------------------------------------------
Right answer: [ 69.88921603 128.         186.11078397 128.           0.
   0.           0.           0.          69.88921603 128.
 1

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.12000000e+02 2.56000000e+02 0.00000000e+00 2.56000000e+02
 7.56589323e-30 7.28799904e-30 0.00000000e+00 1.81438008e-30]
reading folder74
------------------------------------------
Right answer: [126.03901839 126.03901839 252.07803679   0.           3.92196321
   0.           1.96098161   1.96098161   1.96098161   1.96098161
   3.92196321   0.         252.07803679   0.         126.03901839
 126.03901839]
reading folder75
------------------------------------------
Right answer: [34.94460802 93.05539198 64.         64.         34.94460802 93.05539198
 64.         64.         64.         64.         64.         64.
 64.         64.         64.         64.        ]
reading folder76
------------------------------------------
Right answer: [ 68.96414145   7.41165426 103.26573251  61.02401132 129.24833511
  26.69363459  98.44879312  16.94369763 129.24833511

In [10]:
####CS score with standardised answers
def cross_entropy(a,b):
    thershold=10
    candidates = [] # save right answer candidates
    answer = 1
    for results in a:
        candidates.append(np.asarray(results))
    wrong_out = []      
    result=[] 
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)
    for i in range(0, len(a)):
        k = entropy_score(candidates[answer]/candidates[answer].sum(),np.asarray(a[i])/np.asarray(a[i]).sum())-\
        entropy_score(candidates[answer]/candidates[answer].sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


cross_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cross_error.append(["folder"+str(i),cross_entropy(a,b)])
for i in range(0,len(ks_error)):
    if len(cross_error[i][1][1])>0:
        print(cross_error[i][0])

reading folder1
------------------------------------------
Right answer: [256.   0. 256.   0.   0.   0.   0.   0.   0.   0.   0.   0. 256.   0.
 256.   0.]
reading folder2
------------------------------------------
Right answer: [  0.   0.   0.   0. 256.   0. 256.   0. 256.   0. 256.   0.   0.   0.
   0.   0.]
reading folder3
------------------------------------------
Right answer: [256.   0. 256.   0.   0.   0.   0.   0.   0.   0.   0.   0. 256.   0.
 256.   0.]
reading folder4
------------------------------------------
Right answer: [  3.92196321   0.           0.           3.92196321 252.07803679
   0.           0.         252.07803679 252.07803679   0.
   0.         252.07803679   3.92196321   0.           0.
   3.92196321]
reading folder5
------------------------------------------
Right answer: [  0.           0.           0.           0.          69.88921603
 186.11078397  69.88921603 186.11078397  69.88921603 186.11078397
  69.88921603 186.11078397   0.           0.           0.

Right answer: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 512.
 512.   0.]
15.609640474436812
startPyquil_statevector_213.csv:[0.0, 0.0, 0.4999999999999993, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4999999999999993]:15.609640474436812
15.609640474436812
startPyquil_statevector_219.csv:[0.0, 0.0, 0.4999999999999991, 0.0, 0.0, 0.0, 1.540743955509787e-33, 0.0, 0.0, 0.0, 0.0, 1.540743955509787e-33, 0.0, 0.0, 0.0, 0.4999999999999991]:15.609640474436812
15.609640474436812
startPyquil_statevector_223.csv:[0.0, 0.0, 0.49999999999999895, 0.0, 0.0, 0.0, 9.629649721936168e-33, 0.0, 0.0, 0.0, 0.0, 9.629649721936168e-33, 0.0, 0.0, 0.0, 0.49999999999999895]:15.609640474436812
15.609640474436812
startPyquil_statevector_224.csv:[0.0, 0.0, 0.4999999999999991, 0.0, 0.0, 0.0, 4.6607504654171097e-32, 0.0, 0.0, 0.0, 0.0, 4.6607504654171097e-32, 0.0, 0.0, 0.0, 0.4999999999999991]:15.609640474436812
15.609640474436812
startPyquil_statevector_31.csv:[0.0, 0.0, 0.4999

Right answer: [  0.   0.   0.   0.   0. 256.   0. 256.   0. 256.   0. 256.   0.   0.
   0.   0.]
53.5
startPyquil_statevector_220.csv:[4.2730054855919035e-65, 6.162975822039144e-33, 6.162975822039144e-33, 4.2730054855919035e-65, 1.5407439555097865e-33, 0.24999999999999944, 0.24999999999999944, 1.5407439555097865e-33, 1.5407439555097865e-33, 0.24999999999999944, 0.24999999999999944, 1.5407439555097865e-33, 4.2730054855919035e-65, 6.162975822039144e-33, 6.162975822039144e-33, 4.2730054855919035e-65]:53.5
reading folder63
------------------------------------------
Right answer: [ 69.88921603 186.11078397 186.11078397  69.88921603   0.
   0.           0.           0.          69.88921603 186.11078397
 186.11078397  69.88921603   0.           0.           0.
   0.        ]
reading folder64
------------------------------------------
Right answer: [256.   0.   0. 256. 256.   0.   0. 256.   0.   0.   0.   0.   0.   0.
   0.   0.]
53.5
startPyquil_statevector_214.csv:[0.24999999999999978, 0.0, 

Right answer: [34.94460802 93.05539198 93.05539198 34.94460802 34.94460802 93.05539198
 93.05539198 34.94460802 34.94460802 93.05539198 93.05539198 34.94460802
 34.94460802 93.05539198 93.05539198 34.94460802]
reading folder82
------------------------------------------
Right answer: [ 69.88921603 186.11078397   0.           0.           0.
   0.         186.11078397  69.88921603   0.           0.
 186.11078397  69.88921603  69.88921603 186.11078397   0.
   0.        ]
reading folder83
------------------------------------------
Right answer: [ 41.57009369  18.18393886  59.72023178 136.52573567 136.52573567
  59.72023178  18.18393886  41.57009369 136.52573567  59.72023178
  18.18393886  41.57009369  41.57009369  18.18393886  59.72023178
 136.52573567]
reading folder84
------------------------------------------
Right answer: [93.05539198 34.94460802 93.05539198 34.94460802 93.05539198 34.94460802
 93.05539198 34.94460802 93.05539198 34.94460802 93.05539198 34.94460802
 93.05539198 34.9446

In [11]:
##CS score with trimmed mean
def cs_ter(a,b):
    thershold=4
    candidates = [] # save right answer candidates

    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = entropy_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())-\
        entropy_score(np.asarray(candidates)/ np.asarray(candidates).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        k= abs(k)
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

cs_ternn=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cs_ternn.append(["folder"+str(i),cs_ter(a,b)])
for i in range(0,len(ks_error)):
    if len(cs_ternn[i][1][1])>0:
        print(cs_ternn[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00
 3.47098798e-30 0.00000000e+00 3.47098798e-30 0.00000000e+00
 3.47098798e-30 0.00000000e+00 3.47098798e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 2.56000000e+02 0.00000000e+00]
7.304820237218406
startPyquil_statevector_1.csv:[0.2499999999999999, 0.0, 0.0, 0.2499999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2499999999999999, 0.0, 0.0, 0.2499999999999999]:7.304820237218406
7.304820237218406
startPyquil_statevector_222.csv:[0.24999999999999983, 0.0, 0.0, 0.24999999999999983, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24999999999999983, 0.0, 0.0, 0.24999999999999983]:7.304820237218406
7.304820237218406
startPyquil_statevector_223.csv:[0.24999999999999978, 0.0, 0.0, 0.24999999999999978, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24999999999999978, 0.0, 0.0, 0.24999999999999978]:7.304820237218406
reading folder2
---------------------------------

Right answer: [34.94460802 93.05539198 62.06297387 65.93702613 34.94460802 93.05539198
 62.06297387 65.93702613 34.94460802 93.05539198 62.06297387 65.93702613
 34.94460802 93.05539198 62.06297387 65.93702613]
reading folder25
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 0.00000000e+00 2.56000000e+02
 2.28769663e-30 0.00000000e+00 0.00000000e+00 2.28769663e-30
 3.63883438e-30 0.00000000e+00 0.00000000e+00 3.63883438e-30
 2.56000000e+02 0.00000000e+00 0.00000000e+00 2.56000000e+02]
7.304820237218406
startPyquil_statevector_216.csv:[0.2499999999999999, 0.2499999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2499999999999999, 0.2499999999999999, 0.0, 0.0]:7.304820237218406
7.304820237218406
startPyquil_statevector_220.csv:[0.2499999999999999, 0.2499999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2499999999999999, 0.2499999999999999, 0.0, 0.0]:7.304820237218406
7.304820237218406
startPyquil_statevector_227.csv:[0.

Right answer: [34.94460802 93.05539198 58.1889216  69.8110784  34.94460802 93.05539198
 58.1889216  69.8110784  34.94460802 93.05539198 58.1889216  69.8110784
 34.94460802 93.05539198 58.1889216  69.8110784 ]
reading folder47
------------------------------------------
Right answer: [ 69.88921603 124.12594774 186.11078397 131.87405226   0.
   0.           0.           0.          69.88921603 124.12594774
 186.11078397 131.87405226   0.           0.           0.
   0.        ]
reading folder48
------------------------------------------
Right answer: [34.94460802 54.31486934 93.05539198 73.68513066 34.94460802 54.31486934
 93.05539198 73.68513066 34.94460802 54.31486934 93.05539198 73.68513066
 34.94460802 54.31486934 93.05539198 73.68513066]
reading folder49
------------------------------------------
Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.12000000e+02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.73316543e-31
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.733165

Right answer: [5.12000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.83446595e-30 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.83446595e-30 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.12000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
reading folder69
------------------------------------------
Right answer: [ 59.75403255   0.           0.          59.75403255 196.24596745
   0.           0.         196.24596745 196.24596745   0.
   0.         196.24596745  59.75403255   0.           0.
  59.75403255]
6.91286239358257
startPyquil_statevector_210.csv:[0.0583535474078514, 0.0583535474078514, 0.0, 0.0, 0.19164645259214852, 0.19164645259214852, 0.0, 0.0, 0.19164645259214852, 0.19164645259214852, 0.0, 0.0, 0.0583535474078514, 0.0583535474078514, 0.0, 0.0]:6.91286239358257
6.91286239358257
startPyquil_statevector_69.csv:[0.0583535474078514, 0.0583535474078514, 0.0, 0.0, 0.19164645259214852, 0.19164645259214852, 0.0, 0.0, 0.19164645259214852, 0.19164645259214852, 0.0, 0

Right answer: [232.47487486   0.43345844  99.23396275   0.99092435  56.45276772
   0.43345844 120.98962908   0.99092435  56.45276772   0.43345844
 120.98962908   0.99092435 232.47487486   0.43345844  99.23396275
   0.99092435]
reading folder95
------------------------------------------
Right answer: [2.47176417e-30 0.00000000e+00 0.00000000e+00 2.47176417e-30
 2.56000000e+02 0.00000000e+00 0.00000000e+00 2.56000000e+02
 2.56000000e+02 0.00000000e+00 0.00000000e+00 2.56000000e+02
 2.47176417e-30 0.00000000e+00 0.00000000e+00 2.47176417e-30]
7.304820237218406
startPyquil_statevector_210.csv:[0.0, 0.0, 0.0, 0.0, 0.24999999999999978, 0.24999999999999978, 0.0, 0.0, 0.24999999999999978, 0.24999999999999978, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]:7.304820237218406
7.304820237218406
startPyquil_statevector_219.csv:[0.0, 0.0, 0.0, 0.0, 0.24999999999999978, 0.24999999999999978, 0.0, 0.0, 0.24999999999999978, 0.24999999999999978, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]:7.304820237218406
7.304820237218406
startPyqui

In [12]:
#CS score with  2-means


def cs_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=4
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = entropy_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())-\
        entropy_score(np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        k= abs(k)
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


cs_2ms=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cs_2ms.append(["folder"+str(i),cs_2mean(a,b)])
for i in range(0,len(ks_error)):
    if len(cs_2ms[i][1][1])>0:
        print(cs_2ms[i][0])

reading folder1
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 3.33748845e-30 0.00000000e+00 3.33748845e-30 0.00000000e+00
 3.33748845e-30 0.00000000e+00 3.33748845e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02]
reading folder2
------------------------------------------
Right answer: [2.77416085e-30 0.00000000e+00 2.77416085e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 2.77416085e-30 0.00000000e+00 2.77416085e-30 0.00000000e+00]
reading folder3
------------------------------------------
Right answer: [2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02
 7.27903472e-30 0.00000000e+00 7.27903472e-30 0.00000000e+00
 5.16516069e-30 0.00000000e+00 7.27903472e-30 0.00000000e+00
 2.56000000e+02 0.00000000e+00 1.28000000e+02 1.28000000e+02]
reading folder4
------------------------------

Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.15544362e-30 1.26217745e-29
 0.00000000e+00 0.00000000e+00 5.12000000e+02 5.12000000e+02]
reading folder30
------------------------------------------
Right answer: [ 69.88921603 186.11078397 128.         128.           0.
   0.           0.           0.          69.88921603 186.11078397
 128.         128.           0.           0.           0.
   0.        ]
reading folder31
------------------------------------------
Right answer: [0.00000000e+00 0.00000000e+00 2.56000000e+02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.41230079e-29 0.00000000e+00
 0.00000000e+00 2.21768522e-29 2.21768522e-29 2.41230079e-29
 0.00000000e+00 2.56000000e+02 2.56000000e+02 2.56000000e+02]
reading folder32
------------------------------------------
Right answer: [93.05539198 34.94460802 64.         64.         93.05539198 34.94460802
 64.

Right answer: [5.12000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.17655317e-29 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.17655317e-29 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.12000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
reading folder59
------------------------------------------
Right answer: [512.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 512.   0.
   0.   0.]
reading folder60
------------------------------------------
Right answer: [5.1200000e+02 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.7710561e-30
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.7710561e-30 0.0000000e+00
 0.0000000e+00 0.0000000e+00 5.1200000e+02 0.0000000e+00 0.0000000e+00
 0.0000000e+00]
reading folder61
------------------------------------------
Right answer: [2.47498330e-30 1.33441239e-30 7.88860905e-31 3.12918666e-61
 3.72221568e+02 6.98892160e+01 6.98892160e+01 1.05622369e-29
 3.72221568e+02 6.98892160e+01 6.98892160e+01 1.23757700e-29
 2.58135886e-30 2.262614

Right answer: [ 24.44582472  12.22291236   0.          12.22291236 231.55417528
 115.77708764   0.         115.77708764 231.55417528 115.77708764
   0.         115.77708764  24.44582472  12.22291236   0.
  12.22291236]
reading folder88
------------------------------------------
Right answer: [1.52387686e-30 2.47297514e-30 2.50411439e-30 1.40570631e-30
 6.98892160e+01 1.28000000e+02 1.86110784e+02 1.28000000e+02
 6.98892160e+01 1.28000000e+02 1.86110784e+02 1.28000000e+02
 1.52387686e-30 2.47297514e-30 2.50411439e-30 1.40570631e-30]
reading folder89
------------------------------------------
Right answer: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.12000000e+02 0.00000000e+00 2.56000000e+02 2.56000000e+02
 7.88860905e-31 0.00000000e+00 7.88860905e-31 0.00000000e+00]
reading folder90
------------------------------------------
Right answer: [93.05539198 34.94460802 64.         64.         93.05539198 34.94460

In [13]:
###IQR method 

def box_line(a,b):
    def box_line(x):
        a=sorted(x)
        if (len(a)+1)%4==0:

            q1 = a[ int((len(a)+1)*0.25)-1]
            q3 = a[ int((len(a)+1)*0.75)-1]
        else:
            q1 = (a[ int((len(a)+1)*0.25)-1] + a[ math.ceil((len(a)+1)*0.25)-1])/2
            q3 = (a[ int((len(a)+1)*0.75)-1] + a[ math.ceil((len(a)+1)*0.75)-1])/2
            print(q3,q1)
        iqr=q3-q1
        return ((q1-1.5*iqr),(q3+1.5*iqr))

    box_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        box_candidates.append( temp )

    box_answer=[]
    for i in box_candidates:
        box_answer.append(box_line(i))

    print("Right answer:",box_answer)
    wrong_out=[]
    result=[]
    for i in range(0,len(a)):
        flag=0
        for c in range(0,len(a[0])):
            if a[i][c]<box_answer[c][0] or  a[i][c]>box_answer[c][1]:
                flag += 1
        result.append([b[i],flag])
        if flag >= 10:
            print(b[i]+":"+str(a[i])+":"+str(flag))
            wrong_out.append([b[i],flag])
    return result,wrong_out

box_l=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    box_l.append(["folder"+str(i),box_line(a,b)])
for i in range(0,len(ks_ter)):
    if len(box_l[i][1][1])>0:
        print(box_l[i][0])

reading folder1
------------------------------------------
Right answer: [(0.2499999999999984, 0.2500000000000006), (0.0, 0.0), (0.249999999999998, 0.25000000000000067), (0.0, 0.0), (-9.24446373305873e-33, 1.540743955509788e-32), (0.0, 0.0), (-9.24446373305873e-33, 1.540743955509788e-32), (0.0, 0.0), (-9.24446373305873e-33, 1.540743955509788e-32), (0.0, 0.0), (-9.24446373305873e-33, 1.540743955509788e-32), (0.0, 0.0), (0.2499999999999984, 0.2500000000000006), (0.0, 0.0), (0.249999999999998, 0.25000000000000067), (0.0, 0.0)]
reading folder2
------------------------------------------
Right answer: [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.24999999999999806, 0.2500000000000003), (0.0, 0.0), (0.249999999999998, 0.2500000000000002), (0.0, 0.0), (0.24999999999999806, 0.2500000000000003), (0.0, 0.0), (0.249999999999998, 0.2500000000000002), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
reading folder3
------------------------------------------
Right answer: [(0.2499999

Right answer: [(0.058353547407851196, 0.058353547407851473), (0.0, 0.0), (-0.087530321111777, 0.1458838685196283), (-0.087530321111777, 0.1458838685196283), (0.19164645259214777, 0.19164645259214888), (0.0, 0.0), (-0.2874696788882225, 0.4791161314803708), (-0.28746967888822256, 0.4791161314803709), (0.19164645259214777, 0.19164645259214888), (0.0, 0.0), (-0.2874696788882225, 0.4791161314803708), (-0.28746967888822256, 0.4791161314803709), (0.058353547407851196, 0.058353547407851473), (0.0, 0.0), (-0.087530321111777, 0.1458838685196283), (-0.087530321111777, 0.1458838685196283)]
reading folder17
------------------------------------------
Right answer: [(0.24999999999999936, 0.25), (0.0, 0.0), (0.0, 0.0), (0.24999999999999936, 0.25), (-0.37499999999999967, 0.6249999999999994), (0.0, 0.0), (0.0, 0.0), (-0.37499999999999967, 0.6249999999999994), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (-0.3749999999999995, 0.6249999999999991), (0.0, 0.0), (0.0, 0.0), (-0.3749999999999995, 0.6249999

Right answer: [(0.09087440623372124, 0.0908744062337218), (0.03412559376627815, 0.0341255937662784), (0.09087440623372117, 0.09087440623372173), (0.03412559376627815, 0.0341255937662784), (0.09087440623372124, 0.0908744062337218), (0.03412559376627815, 0.0341255937662784), (0.09087440623372117, 0.09087440623372173), (0.03412559376627815, 0.0341255937662784), (0.09087440623372124, 0.0908744062337218), (0.03412559376627815, 0.0341255937662784), (0.09087440623372117, 0.09087440623372173), (0.03412559376627815, 0.0341255937662784), (0.09087440623372124, 0.0908744062337218), (0.03412559376627815, 0.0341255937662784), (0.09087440623372117, 0.09087440623372173), (0.03412559376627815, 0.0341255937662784)]
reading folder33
------------------------------------------
Right answer: [(0.18174881246744268, 0.18174881246744345), (0.06825118753255632, 0.06825118753255677), (-0.10199524986977285, 0.35199524986977215), (-0.10199524986977337, 0.35199524986977304), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0

Right answer: [(0.03412559376627799, 0.03412559376627849), (0.09087440623372095, 0.0908744062337219), (0.03412559376627798, 0.03412559376627851), (0.09087440623372045, 0.09087440623372217), (0.03412559376627799, 0.03412559376627849), (0.09087440623372095, 0.0908744062337219), (0.03412559376627798, 0.03412559376627851), (0.09087440623372045, 0.09087440623372217), (0.03412559376627799, 0.03412559376627849), (0.09087440623372069, 0.09087440623372202), (-0.05099762493488688, 0.1759976249348866), (-0.05099762493488649, 0.1759976249348863), (0.03412559376627799, 0.03412559376627849), (0.09087440623372069, 0.09087440623372202), (-0.05099762493488688, 0.1759976249348866), (-0.05099762493488649, 0.1759976249348863)]
reading folder51
------------------------------------------
Right answer: [(0.2499999999999978, 0.25000000000000067), (0.0, 0.0), (0.24999999999999603, 0.25000000000000155), (0.0, 0.0), (-9.244463733058716e-33, 1.540743955509786e-32), (0.0, 0.0), (-9.244463733058716e-33, 1.540743955

Right answer: [(0.4999999999999979, 0.5000000000000003), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (-2.54222752659115e-32, 4.237045877651917e-32), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (-2.54222752659115e-32, 4.237045877651917e-32), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.4999999999999979, 0.5000000000000003), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
reading folder69
------------------------------------------
Right answer: [(0.058353547407850856, 0.058353547407851605), (0.0, 0.0), (0.0, 0.0), (0.0583535474078508, 0.058353547407851605), (0.19164645259214666, 0.19164645259214932), (0.0, 0.0), (0.0, 0.0), (0.19164645259214652, 0.1916464525921493), (0.19164645259214666, 0.19164645259214932), (0.0, 0.0), (0.0, 0.0), (0.19164645259214635, 0.19164645259214935), (0.058353547407850856, 0.058353547407851605), (0.0, 0.0), (0.0, 0.0), (0.05835354740785076, 0.05835354740785162)]
reading folder70
------------------------------------------
Right answer: [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), 

Right answer: [(0.03412559376627815, 0.0341255937662784), (0.09087440623372131, 0.0908744062337217), (0.03412559376627815, 0.0341255937662784), (0.09087440623372106, 0.09087440623372184), (0.03412559376627815, 0.0341255937662784), (0.09087440623372131, 0.0908744062337217), (0.03412559376627815, 0.0341255937662784), (0.09087440623372106, 0.09087440623372184), (0.03412559376627815, 0.0341255937662784), (0.09087440623372064, 0.09087440623372209), (-0.05099762493488664, 0.17599762493488635), (-0.050997624934886576, 0.1759976249348864), (0.03412559376627815, 0.0341255937662784), (0.09087440623372064, 0.09087440623372209), (-0.05099762493488664, 0.17599762493488635), (-0.050997624934886576, 0.1759976249348864)]
reading folder86
------------------------------------------
Right answer: [(0.034125593766278065, 0.03412559376627837), (0.09087440623372098, 0.09087440623372181), (0.03412559376627802, 0.03412559376627844), (0.09087440623372073, 0.09087440623372195), (0.034125593766278065, 0.03412559

Right answer: [(0.2499999999999988, 0.25000000000000033), (0.0, 0.0), (-0.37499999999999917, 0.6249999999999987), (-0.37499999999999967, 0.6249999999999994), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.2499999999999988, 0.25000000000000033), (0.0, 0.0), (-0.37499999999999917, 0.6249999999999987), (-0.37499999999999967, 0.6249999999999994)]
reading folder98
------------------------------------------
Right answer: [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.24999999999999936, 0.25), (0.0, 0.0), (-0.3749999999999991, 0.6249999999999984), (-0.37499999999999967, 0.6249999999999994), (0.24999999999999936, 0.25), (0.0, 0.0), (-0.3749999999999991, 0.6249999999999984), (-0.37499999999999967, 0.6249999999999994), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
reading folder99
------------------------------------------
Right answer: [(0.06825118753255623, 0.06825118753255678), (0.18174881246744232, 0.18174881246744365), (0.0682511875